In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os, sys

from matplotlib import pyplot as plt
from nltk.corpus import stopwords
from wordcloud import WordCloud


In [4]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import NewsDataLoader
import src.utils as utils

In [16]:
import pandas as pd


def load_data():

        #Loading all the datas into a shared dataframe

    rating_df=pd.read_csv("../data/rating.csv")
    domain_locations_df = pd.read_csv("../data/domains_location.csv")
    traffic_data_df = pd.read_csv("../data/traffic.csv")
        # merge_df=pd.merge(rating_df, domain_locations_df ,left_on='source_name', right_on ='SourceCommonName' ,how ='left')
        # merge_df=pd.merge(merge_df, traffic_data_df , left_on ='source_name' ,right_on='Domain' ,how ='left')
    print("rating")
    print(rating_df.columns)    
    print("domain location")
    print(domain_locations_df.columns)    
    print("traffic")
    print(traffic_data_df.columns)

load_data()

rating
Index(['article_id', 'source_id', 'source_name', 'author', 'title',
       'description', 'url', 'url_to_image', 'published_at', 'content',
       'category', 'article', 'title_sentiment'],
      dtype='object')
domain location
Index(['SourceCommonName', 'location', 'Country'], dtype='object')
traffic
Index(['GlobalRank', 'TldRank', 'Domain', 'TLD', 'RefSubNets', 'RefIPs',
       'IDN_Domain', 'IDN_TLD', 'PrevGlobalRank', 'PrevTldRank',
       'PrevRefSubNets', 'PrevRefIPs'],
      dtype='object')


In [18]:
from src.loader import NewsDataLoader
import pandas as pd

if __name__ == "__main__":
    data_directory = "./data"
    loader = NewsDataLoader(data_directory)

    merge_df = loader.load_data()
    
    # Question 1: Top and Bottom 10 Websites with the Largest Count of News Articles
    top_10_websites = merge_df['source_name'].value_counts().head(10)
    bottom_10_websites = merge_df['source_name'].value_counts().tail(10)

    # Question 2: Websites with the Highest Numbers of Visitors Traffic
    websites_traffic = merge_df.groupby('source_name')['GlobalRank'].max().nlargest(10)

    # Question 3: Countries with the Highest Number of News Media Organisations
    countries_media_organizations = merge_df['Country'].value_counts()

    # Question 4: Countries that have many articles written about them
    articles_about_countries = merge_df['Country'].value_counts()

    # Question 5: Websites that reported about specific countries or regions
    countries_of_interest = ['Africa', 'US', 'China', 'EU', 'Russia', 'Ukraine', 'Middle East']
    websites_reporting_about_countries = merge_df[merge_df['Country'].isin(countries_of_interest)]

    # Question 6: Websites with the highest count of positive, neutral, and negative sentiment
    sentiment_counts_by_website = merge_df.groupby('source_name')['title_sentiment'].value_counts()

    # Question 7: Compare the impact of using mean/average and median
    impact_mean_average_median = merge_df[['sentiment_score']].agg(['mean', 'median'])

    # Question 8: Check the distribution of sentiments for a particular domain
    top_10_domains_by_traffic = merge_df.groupby('source_name')['GlobalRank'].max().nlargest(10)
    sentiment_distribution_for_domain = merge_df[merge_df['source_name'].isin(top_10_domains_by_traffic.index)]['title_sentiment'].value_counts()

    # Question 9: Compare content metadata across sites
    content_metadata_comparison = merge_df.groupby('source_name')[['author', 'category']].nunique()

    # Question 10: Analyze the similarity of message lengths across sites
    message_lengths = merge_df.groupby('source_name')['content'].apply(lambda x: x.str.len().mean())

    # Question 11: Analyze the similarity of the number of words in titles across sites
    title_word_counts = merge_df.groupby('source_name')['title'].apply(lambda x: x.str.split().str.len().mean())

    # Question 12: Analyze the impact of frequent news reporting and sentiment on website's global ranking
    impact_news_reporting_sentiment = merge_df.groupby('source_name')[['article_id', 'sentiment_score', 'GlobalRank']].agg({'article_id': 'count', 'sentiment_score': 'mean', 'GlobalRank': 'mean'})

    # Print or visualize the results as needed
    print("Top 10 Websites with Largest Count of News Articles:\n", top_10_websites)
    print("\nBottom 10 Websites with Smallest Count of News Articles:\n", bottom_10_websites)
    print("\nWebsites with Highest Numbers of Visitors Traffic:\n", websites_traffic)
    # Add print statements or visualizations for other questions as well

FileNotFoundError: [Errno 2] No such file or directory: './data/rating.csv'